In [1]:
import pandas as pd
import re

sentencas = []
textRegex = re.compile(r'text=\"([^\"]+)\"')

with open("../corpus/sentençasSelecionadas.txt", "r", encoding = "utf8") as file:
    for line in file:
        
        if line.find("<s id") >= 0:
            sentencas.append(textRegex.findall(line)[0])
            
sentencas

['Nessa época, Fishel teria conhecido o ginecologista britânico Robert Winston, a quem convidou para um encontro científico promovido por Antinori.',
 'Nenhum filme brasileiro foi escolhido, quebrando assim a expectativa criada em torno de «Alma Corsária» de Carlos Reichenbach e «Mil e Uma» de Suzana de Moraes.',
 'Na terceira colocação, 34 pontos, a Juventus de Turim visita o Genoa, 22, no limiar do rebaixamento.',
 'O projeto vem sendo implementado, em caráter experimental, em três escolas: Morumbi, Visconde de Porto Seguro (particulares) e Godofredo Furtado (estadual).',
 'O cineasta italiano Franco Zefirelli revelou que, para iludir o serviço militar durante o período fascista, se tornou um partisan e acabou matando um soldado alemão.',
 'Mas é também preciso a sociedade mobilizar-se para sensibilizar o futuro Congresso Nacional na promoção de reformas constitucionais indispensáveis ao êxito das mudanças pretendidas.',
 'Na Anhanguera, que une São Paulo a Ribeirão Preto, costuma ha

In [2]:
import ufal.udpipe
# ufal.udpipe.Model etc. are SWIG-magic and cannot be detected by pylint
# pylint: disable=no-member

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [4]:
model = Model('portuguese-br-ud-2.0-conll17-170315.udpipe')

In [12]:
for frase in sentencas:
    sentences = model.tokenize(frase)

    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu = model.write(sentences, "conllu")
    lista = conllu.split('#')[4].split('\n')[1:-2]
    table = []
    columns = ['Id','Form','Lemma','UPosTag','XPosTag','Feats','Head','DepRel','Deps','Misc']
    for li in lista:
        table.append(li.split('\t'))

    udpipePos = pd.DataFrame(data=table, columns=columns)
    print(udpipePos[['Form','UPosTag','XPosTag']])
    print('---')

             Form UPosTag XPosTag
0           Nessa     ADP     ADP
1           época    NOUN    NOUN
2               ,   PUNCT       .
3          Fishel   PROPN   PNOUN
4           teria     AUX     AUX
5       conhecido    VERB    VERB
6               o     DET     DET
7   ginecologista    NOUN    NOUN
8       britânico     ADJ     ADJ
9          Robert   PROPN   PNOUN
10        Winston   PROPN   PNOUN
11              ,   PUNCT       .
12              a     ADP     ADP
13           quem    PRON    PRON
14       convidou    VERB    VERB
15           para     ADP     ADP
16             um     DET     DET
17       encontro    NOUN    NOUN
18     científico     ADJ     ADJ
19      promovido    VERB    VERB
20            por     ADP     ADP
21       Antinori   PROPN   PNOUN
22              .   PUNCT       .
---
           Form UPosTag XPosTag
0        Nenhum     DET     DET
1         filme    NOUN    NOUN
2    brasileiro     ADJ     ADJ
3           foi     AUX     AUX
4     escolhido    V

In [ ]:
# Universal Dependecy POS List : http://universaldependencies.org/u/pos/index.html

#ADJ: adjective
#ADP: adposition
#ADV: adverb
#AUX: auxiliary
#CCONJ: coordinating conjunction
#DET: determiner
#INTJ: interjection
#NOUN: noun
#NUM: numeral
#PART: particle
#PRON: pronoun
#PROPN: proper noun
#PUNCT: punctuation
#SCONJ: subordinating conjunction
#SYM: symbol
#VERB: verb
#X: other
